In [613]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt



In [614]:
names = open("./names.txt", 'r').read().split()

In [615]:
len(names)

32033

In [616]:
print(names[0])

emma


In [617]:
import string 

stoi = {l:i for i,l in enumerate('.' + string.ascii_lowercase)}
itos = {l:i for i,l in stoi.items()}

In [618]:
stoi

{'.': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [581]:
from torch import float32


def create_dataset(words):
    X = []
    Y = []
    for name in words:
        name =name
        for i,letter in enumerate(name[:len(name)-1]):
            x_i = [0] * 27
            y_i = [0] * 27
            letter_idx = stoi[letter]
            target_idx = stoi[name[i+1]]
            x_i[letter_idx] = 1
            y_i[target_idx] = 1
            X.append(x_i)
            Y.append(y_i)
    X = torch.tensor(X, dtype=float32).to("cuda")
    print(X.shape)
    Y = torch.tensor(Y, dtype=float32).to("cuda")
    print(Y.shape)
    return X, Y
            


In [761]:
import random

random.seed(42)
random.shuffle(names)
n1 = int(0.8 * len(names))
n2 = int(0.9 * len(names))

x_tr, y_tr = create_dataset(names[:10000])
x_dev, y_dev= create_dataset(names[n1:n2])
x_te, y_te = create_dataset(names[n2:])

torch.Size([51460, 27])
torch.Size([51460, 27])
torch.Size([16249, 27])
torch.Size([16249, 27])
torch.Size([16458, 27])
torch.Size([16458, 27])


In [628]:
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [770]:
#hyperparameters
hidden_size = 100
seq_length = 4
lr = 0.1
batch_size = round((len(x_tr) /seq_length)+0.5) # = math.ceil


In [771]:
rnn = nn.RNN(27, hidden_size)
rnn.cuda()
linear1 = nn.Linear(hidden_size, 27)
linear1.cuda()


Linear(in_features=100, out_features=27, bias=True)

In [772]:
from torch.nn.functional import linear


Wxh = rnn.weight_ih_l0.clone().detach().cuda()
Whh = rnn.weight_hh_l0.clone().detach().cuda()
bh = rnn.bias_hh_l0.clone().detach().cuda() 
bx = rnn.bias_ih_l0.clone().detach().cuda() 

Why = linear1.weight.clone().detach().cuda()
by = linear1.bias.clone().detach().cuda()


In [765]:
out, h_n = rnn(x_tr[0].unsqueeze(0))
logits = linear1(out)
loss = F.cross_entropy(logits, y_tr[0].unsqueeze(0))
print(loss)

tensor(3.3545, device='cuda:0', grad_fn=<DivBackward1>)


In [713]:
logits.retain_grad()
for p in rnn.parameters():
    p.retain_grad()

for p in linear1.parameters():
    p.retain_grad()
loss.retain_grad()


In [714]:
loss.backward()

In [730]:
for name, p in rnn.named_parameters():
    print(name)
    print(p.grad)


weight_ih_l0
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
weight_hh_l0
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
bias_ih_l0
tensor([ 0.0780, -0.0220, -0.0328,  0.0320,  0.0151, -0.0860,  0.0567,  0.0668,
         0.1011,  0.0412, -0.0727, -0.0556, -0.0545, -0.0491, -0.0220, -0.0551,
         0.0400, -0.0831, -0.1060,  0.0521,  0.0686,  0.1019,  0.0633,  0.0219,
         0.0310, -0.0883,  0.0541,  0.0714, -0.0706,  0.0204, -0.0187,  0.0064,
         0.0407, -0.0594,  0.0012,  0.0754,  0.0340,  0.0415,  0.0702, -0.0703,
         0.0810, -0.0

In [773]:
n, p = 0, 0
mWxh, mWhh, mWhy = torch.zeros_like(Wxh), torch.zeros_like(Whh), torch.zeros_like(Why)
mbh, mby = torch.zeros_like(bh), torch.zeros_like(by) # memory variables for Adagrad
#smooth_loss = -torch.log(1.0/27.0)*seq_length # loss at iteration 0
#smooth_loss = smooth_loss * 0.999 + loss * 0.001
#print('iter %d, loss: %f' % (n, smooth_loss)) # print progress

In [774]:
print(bh.shape)

torch.Size([100])


In [775]:
#forward 
def forward(x, y, h_prev):
    xs, hs, ys, ps = {},{},{},{}
    loss = 0.0
    hs[-1] = h_prev.clone()
    for t in range(len(x)):
        xs[t] = x[t].unsqueeze(0)
        hs[t] = torch.tanh(xs[t] @ Wxh.T + bx + hs[t-1] @ Whh.T  + bh)
        ys[t] = hs[t] @ Why.T + by
        ps[t] = torch.exp(ys[t]) / torch.sum(torch.exp(ys[t])) 
        loss += -torch.log(ps[t][0,y[t].argmax().item()])
    return loss, ps, ys, hs, xs

def backward(ps, x, y, hs, xs):
    #backward in time
    dWxh, dWhh, dWhy = torch.zeros_like(Wxh), torch.zeros_like(Whh), torch.zeros_like(Why), 
    dbh, dby = torch.zeros_like(bh), torch.zeros_like(by)
    dhnext = torch.zeros_like(hs[0])
    for t in reversed(range(len(x))):
        dy = ps[t].clone()
        dy[0, y[t].argmax().item()] -= 1
        dWhy +=  dy.T @ hs[t] 
        dby += dy.sum(0)
        # ------ below not tested ----
        dh = dy @ Why + dhnext
        dhraw = (1 - hs[t] * hs[t]) * dh
        dbh = dhraw.sum(0)
        dWxh +=  dhraw.T @ xs[t]  
        dWhh += hs[t-1] @ dhraw.T  
        dhnext = dhraw @ Whh.T  

    for dparam in [dWxh, dWhh, dWhy, dbh, dby, hs[len(x)-1]]:
        torch.clip(dparam, -5, 5, out=dparam)
    return dWxh, dWhh, dWhy, dbh, dby


In [776]:
h_prev = torch.zeros(hidden_size).to("cuda")
lossme, ps, ys, hs, xs = forward(x_tr[0].unsqueeze(0), y_tr[0].unsqueeze(0), h_prev)
dWxh, dWhh, dWhy, dbh, dby = backward(ps, x_tr[0].unsqueeze(0),y_tr[0].unsqueeze(0), hs, xs) 

In [777]:
print(loss)
#cmp("Wxh", dWxh, linear1.weight_hh_l0)

tensor(61.1033, device='cuda:0')


In [778]:

# memory variables for Adagrad
mWxh, mWhh, mWhy = torch.zeros_like(Wxh), torch.zeros_like(Whh), torch.zeros_like(Why)
mbh, mby = torch.zeros_like(bh), torch.zeros_like(by) 

epoch=1000
for i in range(epoch):
    h_prev = torch.zeros(1, hidden_size).to("cuda")
    for p in range(0, x_tr.shape[0], seq_length):
        # forward
        inputs = x_tr[p:p+seq_length]
        targets= y_tr[p:p+seq_length]

        loss, ps,ys, hs, xs = forward(inputs, targets, h_prev)

        # backward
        for param in [Wxh, Whh, Why, bh, by]:
            param.grad = None
        dWxh, dWhh, dWhy, dbh, dby = backward(ps, inputs,targets, hs, xs) 
        
        
    # perform parameter update with Adagrad
        for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                    [dWxh, dWhh, dWhy, dbh, dby], 
                                    [mWxh, mWhh, mWhy, mbh, mby]):
            mem += dparam * dparam # elementwise
            param += -lr* dparam / torch.sqrt(mem + 1e-8) # adagrad update      
        
    if i % 10 == 0:
        print ('iter %d, loss: %f' % (i, loss)) # print progress


iter 0, loss: 8.940229


KeyboardInterrupt: 

In [779]:
import numpy as np
def predict(test_char, length):
    x = torch.zeros((1, 27)) 
    x[0,stoi[test_char]] = 1
    ixes = []
    h = torch.zeros((1,hidden_size))
    wxh = Wxh.clone().cpu()
    whh = Whh.clone().cpu()
    why = Why.clone().cpu()
    b = bh.clone().cpu()
    bb = by.clone().cpu()

    for t in range(length):
        h = torch.tanh(x @ wxh.T + h @ whh.T  + b)
        y = h @ why.T + bb
        p = torch.exp(y) / torch.sum(torch.exp(y)) 
        ix = np.random.choice(range(27), p=p.numpy().ravel()) # ravel -> rank0
        # "ix" is a list of indexes selected according to the soft max probability.
        x = torch.zeros((1,27)) # init
        x[0,ix] = 1 
        ixes.append(ix) # list
    txt = ''.join(itos[i] for i in ixes)
    print ('----\n %s \n----' % (txt, ))

In [780]:
predict('a',10)

----
 rejauaaole 
----


In [760]:
for i in range(26):
    for j in range(3,10):
        predict(itos[i],j)

----
 ior 
----
----
 elie 
----
----
 elyna 
----
----
 arisel 
----
----
 encered 
----
----
 eelliann 
----
----
 emaleelan 
----
----
 nna 
----
----
 nnav 
----
----
 ionsa 
----
----
 ishtia 
----
----
 ndieyan 
----
----
 ykataiqu 
----
----
 saiqusemy 
----
----
 emy 
----
----
 esad 
----
----
 esstl 
----
----
 esayna 
----
----
 etleval 
----
----
 esslayre 
----
----
 essllyana 
----
----
 eyl 
----
----
 harl 
----
----
 aszal 
----
----
 haista 
----
----
 hancera 
----
----
 hriyeras 
----
----
 hanaskyle 
----
----
 eny 
----
----
 anda 
----
----
 yiaha 
----
----
 aishio 
----
----
 ancersi 
----
----
 ayeiyorl 
----
----
 ielledgan 
----
----
 myi 
----
----
 liai 
----
----
 lysin 
----
----
 idalda 
----
----
 llukasi 
----
----
 laencers 
----
----
 anasslayo 
----
----
 rei 
----
----
 rely 
----
----
 ranna 
----
----
 reyans 
----
----
 oparqus 
----
----
 aidiland 
----
----
 rannavish 
----
----
 aro 
----
----
 arab 
----
----
 arsey 
----
----
 elinab 
----

In [521]:
h_prev

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]], device='cuda:0')